# Guide avanzate OpenAI assistants

## Introduzione

Questo notebook servirà ad introdurre e a spiegare alcuni elementi più avanzati sugli strumenti messi a disposizione da OpenAI. E' raccomandata la lettura dei notebooks "RAG" e "OpenAI Assistants Base" prima di questo.    

Ci saranno diverse sezioni (perlopiù indipendenti una dall'altra) in cui verranno introdotti argomenti diversi.

Gli argomenti trattati saranno:

- Streaming: come mostrare all'utente in tempo reale le parole mentre vengono generate dall'Assistant
- File Annotations: come utilizzare i riferimenti che compaiono all'interno delle risposte di un'Assistant, e come farle puntare al documento.
- Function Calling


In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

client = OpenAI(api_key=os.getenv("OPENAI_KEY"))

## Streaming

Nel notebook base abbiamo visto come ottenere le risposte in modalità sincrona: questo significa che noi (e l'utente finale) potremo vedere la risposta soltanto quando essa ha terminato di essere "assemblata" dal modello.  
  
Con lo streming invece il testo fluisce in chunks direttamente mentre il modello lo crea. Con l'SDK base di OpenAI lo streaming viaggia sotto forma di chunks che arrivano seguendo lo standard SSE (Server Sent Events). Noi possiamo scegliere di accumulare questi chunks fino a quando arrierà l'ultimo, che chiameremo stop-chunk.

Penso sia utile vedere in apertura come funziona lo streaming nell'SDK classico di OpenAI.

In [2]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Spiegami la teoria del multiverso"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

La teoria del multiverso è una ipotesi fisica che suggerisce l'esistenza di un numero infinito di universi diversi dal nostro, ognuno con le proprie leggi fisiche e costanti fondamentali. Secondo questa teoria, ogni possibile universo che potrebbe esistere, esiste effettivamente in un certo punto del multiverso.

Questa ipotesi è stata proposta per spiegare alcune delle inconsistenze e delle apparenti contraddizioni nella fisica moderna, come ad esempio il concetto di universo inflazionario, il paradosso di Schrödinger e il problema dei costanti fondamentali. La teoria del multiverso suggerisce che, anziché cercare una spiegazione unificata per tutti questi fenomeni, potrebbero esistere molteplici universi che si comportano in modi diversi.

È importante sottolineare che la teoria del multiverso è ancora oggetto di dibattito tra gli scienziati e non è attualmente provata sperimentalmente. Tuttavia, questa ipotesi offre un'interessante prospettiva sulla natura dell'universo e ha stimola

Possiamo vedere che ha impiegato un certo tempo, ma comunque i chunks sono spuntati correttamente.  
Lo streaming inizia sempre con un chunk vuoto, e finisce sempre con uno stop chunk, riconoscibile per la finish_reason (chunk.choices[0].finish_reason) non nulla (solitamente "stop")  
La struttura di un singolo chunk è:
```json
{"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0125", "system_fingerprint": "fp_44709d6fcb", "choices":[{"index":0,"delta":{"role":"assistant","content":""},"logprobs":null,"finish_reason":null}]}

{"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0125", "system_fingerprint": "fp_44709d6fcb", "choices":[{"index":0,"delta":{"content":"Hello"},"logprobs":null,"finish_reason":null}]}

....

{"id":"chatcmpl-123","object":"chat.completion.chunk","created":1694268190,"model":"gpt-3.5-turbo-0125", "system_fingerprint": "fp_44709d6fcb", "choices":[{"index":0,"delta":{},"logprobs":null,"finish_reason":"stop"}]}
```


In sede di produzione, basta stabilire una connessione asincrona con il frontend (WebSocket solitamente) e effettuare il mirroring dei chunks man mano che arrivano, chiudendo il messaggio quando arriva lo stop-chunk

### Streaming con OpenAI Assistants

Quando lavoriamo con gli assistants di OpenAI abbiamo sempre la possibilità di utilizzare lo streaming, anche in ragione del fatto che il tempo di generazione per una risposta è solitamente maggiore rispetto ad una chiamata classica ai modelli.  

In questo caso, lo streaming arriverà attraverso l'oggetto che istanziamo quando eseguiamo il RUN sul thread. Useremo (come al solito) l'assistant con la documentazione di IVIC.  

Iniziamo: creiamo un thread e inseriamo un messaggio user con una domanda.



In [26]:
IVIC_ASST = "asst_RUodZv7nz1qutEcGB6a1z8Do"
ivic_thread = client.beta.threads.create()
client.beta.threads.messages.create(
                thread_id=ivic_thread.id,                          
                role="user",                                        
                content="Cosa si intende per domanda di disambiguazione?"  
)

Message(id='msg_WNadRumwkmStfH5L90yGFNwt', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Cosa si intende per domanda di disambiguazione?'), type='text')], created_at=1718095285, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_e1Jb6NUjJ56Rg4RrROs8Ngud')

Per funzionare, un run con streaming ha bisogno di una classe EventHandler, che specifica le azioni che il nostro runtime dovrà eseguire quando si verificano determinati eventi nel flusso di streaming che ci arriva.

Vediamo ora un EventHandler base, che si limita a dire quando viene effettuata la ricerca nel vectorstore, e mi fa lo streaming del solo testo.  

Quua sotto possiamo vedere il funzionamento di un semplice eventHandler base:

In [27]:
# importo il decoratore che mi servità per sovrascrivere i metodi già presenti in runtime
from typing_extensions import override  

# importo la superclasse di event handler
from openai import AssistantEventHandler

class EventHandler(AssistantEventHandler): 
  
  # metodo invocato quando il messaggio inizia, e che aggiunge una piccola intestazione al messaggio ("Ivic_assistente >").
  # se volessi fare operazioni sul contenuto del messaggio nascituro, devo azionarle sulla variabile locale "text"
  @override
  def on_text_created(self, text) -> None: 
    print(f"\nIvic_assistente > ", end="", flush=True)
    
  # metodo invocato ad ogni nuovo chunk
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
    
  # metodo invocato quando un tool (file_search o code_intepreter) esegue un'azione
  def on_tool_call_created(self, tool_call):
    print(f"\nEvento > {tool_call.type}\n", flush=True)


In [28]:
with client.beta.threads.runs.stream(
  thread_id=ivic_thread.id,
  assistant_id=IVIC_ASST,
 event_handler=EventHandler(),
) as stream:
  stream.until_done()


Evento > file_search


Ivic_assistente > Una domanda di disambiguazione in un software come ivic è una richiesta posta all'utente per chiarire e descrivere brevemente una transazione che potrebbe non essere stata identificata in modo chiaro come parte di un determinato schema di frode. Se le regole di identificazione non sono state sufficienti a individuare con sicurezza un pattern di frode specifico, viene posta una domanda di disambiguazione per ottenere maggiori dettagli sulla transazione dall'utente. La risposta dell'utente viene poi analizzata tramite un algoritmo di intelligenza artificiale che incrocia le informazioni fornite dall'utente con le descrizioni ufficiali dei modelli di frode per determinare se la transazione potrebbe rientrare in un determinato schema. Basandosi su questa analisi, il software può produrre diversi esiti, tra cui la declinazione della transazione o la conferma di validità della transazione【4:0†source】【4:2†source】.